In [11]:
import torch
import numpy as np 
import pandas as pd 
from torch import nn
import matplotlib.pyplot as plt


In [14]:
weight = 0.7
bias = 0.3

start = 0
end = 1
step = 0.02
X = torch.arange(start, end, step).unsqueeze(dim=1)
y = weight*X + bias
X[:10], y[:10]

(tensor([[0.0000],
         [0.0200],
         [0.0400],
         [0.0600],
         [0.0800],
         [0.1000],
         [0.1200],
         [0.1400],
         [0.1600],
         [0.1800]]),
 tensor([[0.3000],
         [0.3140],
         [0.3280],
         [0.3420],
         [0.3560],
         [0.3700],
         [0.3840],
         [0.3980],
         [0.4120],
         [0.4260]]))

In [15]:
len(X), len(y)

(50, 50)

In [22]:
#train split 
train_split = int(0.8 * len(X))
X_train, y_train = X[: train_split], y[:train_split]
X_test, y_test = X[train_split:], y[train_split:]
len(X_train), len(X_test) 

(40, 10)

In [36]:
def plot_predictions(train_data = X_train,
                     train_labels = y_train,
                     test_data = X_test,
                     test_label = y_test,
                     predictions =None):
    plt.figure(figsize=(10,7))
    plt.scatter(train_data,train_labels,c='b', s=4, label="training data")
    plt.scatter(test_data,test_label,c='g', s=4, label="test data")
    if predictions is not None:
            plt.scatter(test_data,predictions,c='r', s=4, label="predictions")

    